In [ ]:

import numpy as np
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
import cv2
import numpy as np
import pandas as pd
import random,os,glob
from sklearn.utils import shuffle
from urllib.request import urlopen
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [4]:
train_dir='/kaggle/input/gender-classification-dataset/Training/'
test_dir='/kaggle/input/gender-classification-dataset/Validation/'
    

In [5]:
import cv2
import os
import pandas as pd



def get_paths(directory):
    image_paths = []
    labels = []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image_paths.append(image_path)
                labels.append(label)
    return image_paths, labels



img_train_paths, lbl_train = get_paths(train_dir)
img_test_paths, lbl_test = get_paths(test_dir)











    


In [6]:
train_df=pd.DataFrame({'path':img_train_paths,'gender':lbl_train})

test_df=pd.DataFrame({'path':img_test_paths,'gender':lbl_test})

In [7]:
train_df

,path,gender
0,/kaggle/input/gender-classification-dataset/Tr...,female
1,/kaggle/input/gender-classification-dataset/Tr...,female
2,/kaggle/input/gender-classification-dataset/Tr...,female
3,/kaggle/input/gender-classification-dataset/Tr...,female
4,/kaggle/input/gender-classification-dataset/Tr...,female
...,...,...
47004,/kaggle/input/gender-classification-dataset/Tr...,male
47005,/kaggle/input/gender-classification-dataset/Tr...,male
47006,/kaggle/input/gender-classification-dataset/Tr...,male
47007,/kaggle/input/gender-classification-dataset/Tr...,male


In [8]:
test_df

,path,gender
0,/kaggle/input/gender-classification-dataset/Va...,female
1,/kaggle/input/gender-classification-dataset/Va...,female
2,/kaggle/input/gender-classification-dataset/Va...,female
3,/kaggle/input/gender-classification-dataset/Va...,female
4,/kaggle/input/gender-classification-dataset/Va...,female
...,...,...
11644,/kaggle/input/gender-classification-dataset/Va...,male
11645,/kaggle/input/gender-classification-dataset/Va...,male
11646,/kaggle/input/gender-classification-dataset/Va...,male
11647,/kaggle/input/gender-classification-dataset/Va...,male


In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
lblen=  label_encoder.fit_transform(train_df['gender'])
lblen_ts=label_encoder.fit_transform(test_df['gender'])





In [10]:
train_df['label']=lblen
test_df['label']=lblen_ts

train_df=shuffle(train_df)
test_df=shuffle(test_df)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)





In [11]:
test_df

,path,gender,label
0,/kaggle/input/gender-classification-dataset/Va...,male,1
1,/kaggle/input/gender-classification-dataset/Va...,male,1
2,/kaggle/input/gender-classification-dataset/Va...,male,1
3,/kaggle/input/gender-classification-dataset/Va...,male,1
4,/kaggle/input/gender-classification-dataset/Va...,female,0
...,...,...,...
11644,/kaggle/input/gender-classification-dataset/Va...,female,0
11645,/kaggle/input/gender-classification-dataset/Va...,female,0
11646,/kaggle/input/gender-classification-dataset/Va...,female,0
11647,/kaggle/input/gender-classification-dataset/Va...,male,1


In [12]:
train_df

,path,gender,label
0,/kaggle/input/gender-classification-dataset/Tr...,male,1
1,/kaggle/input/gender-classification-dataset/Tr...,female,0
2,/kaggle/input/gender-classification-dataset/Tr...,female,0
3,/kaggle/input/gender-classification-dataset/Tr...,female,0
4,/kaggle/input/gender-classification-dataset/Tr...,male,1
...,...,...,...
47004,/kaggle/input/gender-classification-dataset/Tr...,male,1
47005,/kaggle/input/gender-classification-dataset/Tr...,female,0
47006,/kaggle/input/gender-classification-dataset/Tr...,female,0
47007,/kaggle/input/gender-classification-dataset/Tr...,male,1


In [14]:
img=cv2.imread(train_df.path[5])
img.shape


(103, 81, 3)

### 

In [15]:
datagen=ImageDataGenerator(rescale=1./255,
                           rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    shear_range=0.1,  
    zoom_range=0.1, 
    horizontal_flip=True,  
    fill_mode='nearest'  
                )

batch_size=128

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="path",
    y_col="gender",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='binary'

)

test_generator=datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="path",
    y_col="gender",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='binary'

)

Found 47009 validated image filenames belonging to 2 classes.
Found 11649 validated image filenames belonging to 2 classes.


In [ ]:
train_data_size = len(train_generator.classes)
test_data_size = len(test_generator.classes)


model = Sequential()



model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))



model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', mode='min' ,patience=5)
model_checkpoint = ModelCheckpoint('bestmodel.h5', mode='min',monitor='val_loss', save_best_only=True)


history = model.fit(
    train_generator,
    steps_per_epoch=train_data_size // batch_size,
    epochs=35,
    validation_data=test_generator,
    validation_steps=test_data_size // batch_size,
    callbacks=[early_stopping, model_checkpoint]  
)


In [430]:
train_df.path[0]

'resized_195645.jpg.jpg'

In [416]:
best_model = load_model('bestmodel.h5')
best_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 78, 78, 128)       3584      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 39, 39, 128)      0         
 g2D)                                                            
                                                                 
 dropout_55 (Dropout)        (None, 39, 39, 128)       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 37, 37, 256)       295168    
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 18, 18, 256)      0         
 g2D)                                                            
                                                                 
 dropout_56 (Dropout)        (None, 18, 18, 256)     

In [5]:

import tensorflow as tf


new_model = tf.keras.models.load_model('bestmodel.h5')



In [7]:


gender_labels = ["Kadin", "Erkek"]


cap = cv2.VideoCapture(0)


girdi_genislik = new_model.input_shape[1]
girdi_yukseklik = new_model.input_shape[2]

while True:
   
    ret, frame = cap.read()

  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3, minSize=(30, 30))

  
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        
        face = cv2.resize(face, (girdi_genislik, girdi_yukseklik))

        
        face = face.astype("float32") / 255.0

       
        face = np.expand_dims(face, axis=0)

        prediction = new_model.predict(face)[0]
        gender_index = 1 if prediction[0] >= 0.5 else 0
        gender_label = gender_labels[gender_index]
        
        cv2.putText(frame, gender_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

   
    cv2.imshow('Cinsiyet Tahmini', frame)


cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step
